# 인구분포

In [ ]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


In [ ]:
import requests
import zipfile


def save_file(url, file_name):
  r = requests.get(url)
  with open(file_name, 'wb') as f:
    f.write(r.content)

    
save_file('https://courses.cs.washington.edu/courses/cse163/19sp/' +
          'files/lectures/05-13/data.zip', 'data.zip')
save_file('https://courses.cs.washington.edu/courses/cse163/19sp/' +
          'files/lectures/05-13/gz_2010_us_040_00_5m.json', 
          'gz_2010_us_040_00_5m.json')
save_file('https://courses.cs.washington.edu/courses/cse163/19sp/' +
          'files/lectures/05-13/stormhistory.csv', 'stormhistory.csv')

with zipfile.ZipFile("data.zip","r") as zip_ref:
    zip_ref.extractall()
    
!pip install --upgrade geopandas
!pip install --upgrade pyshp
!pip install --upgrade shapely
!pip install --upgrade descartes

Requirement already up-to-date: geopandas in /usr/local/lib/python3.6/dist-packages (0.8.1)
Requirement already up-to-date: pyshp in /usr/local/lib/python3.6/dist-packages (2.1.2)
Requirement already up-to-date: shapely in /usr/local/lib/python3.6/dist-packages (1.7.1)
Requirement already up-to-date: descartes in /usr/local/lib/python3.6/dist-packages (1.1.0)


In [ ]:
import pathlib
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, Point
from pandas import Series, DataFrame
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings(action='ignore')


In [ ]:
인구분포도 = gpd.read_file('/gdrive/My Drive/광양시 데이터/08.광양시_격자별인구현황(100X100).geojson')
인구분포도.loc[인구분포도['val'].apply(pd.isna), 'val'] = 0
인구분포도

,gid,val,geometry
0,라라023745,0.0,"MULTIPOLYGON (((127.52523 35.06557, 127.52523 ..."
1,라라023746,0.0,"MULTIPOLYGON (((127.52523 35.06647, 127.52523 ..."
2,라라023747,0.0,"MULTIPOLYGON (((127.52523 35.06737, 127.52523 ..."
3,라라023748,0.0,"MULTIPOLYGON (((127.52523 35.06828, 127.52523 ..."
4,라라023749,0.0,"MULTIPOLYGON (((127.52523 35.06918, 127.52523 ..."
...,...,...,...
47027,라라261688,0.0,"MULTIPOLYGON (((127.78607 35.01384, 127.78607 ..."
47028,라라261689,0.0,"MULTIPOLYGON (((127.78607 35.01474, 127.78608 ..."
47029,라라261690,0.0,"MULTIPOLYGON (((127.78608 35.01564, 127.78608 ..."
47030,라라261691,0.0,"MULTIPOLYGON (((127.78608 35.01654, 127.78608 ..."


In [ ]:
인구분포도.describe()

,val
count,47032.000000
mean,3.097529
std,36.054919
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,2613.000000


In [ ]:
#인구가 0인곳이 45000개 정도
인구분포도[인구분포도.val==0]

,gid,val,geometry
0,라라023745,0.0,"MULTIPOLYGON (((127.52523 35.06557, 127.52523 ..."
1,라라023746,0.0,"MULTIPOLYGON (((127.52523 35.06647, 127.52523 ..."
2,라라023747,0.0,"MULTIPOLYGON (((127.52523 35.06737, 127.52523 ..."
3,라라023748,0.0,"MULTIPOLYGON (((127.52523 35.06828, 127.52523 ..."
4,라라023749,0.0,"MULTIPOLYGON (((127.52523 35.06918, 127.52523 ..."
...,...,...,...
47027,라라261688,0.0,"MULTIPOLYGON (((127.78607 35.01384, 127.78607 ..."
47028,라라261689,0.0,"MULTIPOLYGON (((127.78607 35.01474, 127.78608 ..."
47029,라라261690,0.0,"MULTIPOLYGON (((127.78608 35.01564, 127.78608 ..."
47030,라라261691,0.0,"MULTIPOLYGON (((127.78608 35.01654, 127.78608 ..."


In [ ]:
# 인구 0 제외
인구분포도2 = 인구분포도[인구분포도['val']!=0]
인구분포도2.describe()

,val
count,2035.000000
mean,71.588698
std,158.595558
min,6.000000
25%,9.000000
50%,16.000000
75%,46.500000
max,2613.000000


In [ ]:
축척 = sum([
    *(인구분포도['geometry'].bounds['maxx'] - 인구분포도['geometry'].bounds['minx']),
    *(인구분포도['geometry'].bounds['maxy'] - 인구분포도['geometry'].bounds['miny']),
]) / (2 * len(인구분포도)) / 100
축척

1.0004932768754573e-05

In [ ]:
#차 속도 = 60km/h
#15분 이내의 충전소에 간다고 가정 --> 15분에 15km (15000m)
속도 = 60
거리 = 60*1000/4
거리

15000.0

In [ ]:
주차장 = pd.read_excel('/gdrive/My Drive/광양시 데이터/chargingStation.xlsx')

In [ ]:
# 전체 인구분포로 계산
인구분포도_경위도 = np.stack([인구분포도['geometry'].centroid.x, 인구분포도['geometry'].centroid.y], axis=1).reshape([1, -1, 2])

for i in range(len(주차장)):
  주차장_경위도 = np.stack([주차장.lon,주차장.lat], axis=1).reshape([-1,1,2])
  인구분포_조건 = np.sum((주차장_경위도 - 인구분포도_경위도)**2, axis=2)**0.5 < 거리*축척
  인구분포 = np.array([인구분포도.loc[x,'val'].mean() for x in 인구분포_조건])

In [ ]:
인구분포

array([4.13780676, 3.94374436, 3.94284392, 4.07910539, 3.98849976,
       4.26979216, 3.87311472, 3.94010059, 4.03072882, 4.09215835,
       4.09309705, 3.56636364, 4.10421232, 4.06164557, 4.18886185,
       4.11463436, 4.05412107, 4.04064738, 3.96358559, 4.18519484,
       4.05768534, 3.7108969 , 3.7108969 , 3.63353363, 3.63353363,
       4.07288864, 4.12461434, 4.12461434, 4.35375053, 4.32416891,
       4.26915915, 4.46139546, 3.99641122, 4.27896233, 4.27896233,
       4.37441247, 4.34891425, 4.3175405 , 3.93060682, 3.66888111,
       3.66888111, 3.734696  , 3.734696  , 3.93369842])

In [ ]:
#인구 0인 곳 제외하고 계산
인구분포도_경위도2 = np.stack([인구분포도2['geometry'].centroid.x, 인구분포도2['geometry'].centroid.y], axis=1).reshape([1, -1, 2])
for i in range(len(주차장)):
  주차장_경위도 = np.stack([주차장.lon,주차장.lat], axis=1).reshape([-1,1,2])
  인구분포_조건2 = np.sum((주차장_경위도 - 인구분포도_경위도2)**2, axis=2)**0.5 < 거리*축척
  인구분포2 = np.array([인구분포도2.loc[x,'val'].mean() for x in 인구분포_조건2])

In [ ]:
인구분포2

array([83.42906574, 81.48586891, 74.44575835, 74.91688865, 78.14635807,
       83.75965665, 82.64132104, 83.29261863, 75.53405839, 83.01415385,
       75.87902331, 73.16078629, 83.13686806, 74.81282316, 75.09922078,
       75.39989119, 74.82956339, 82.79733164, 74.54479918, 75.0987013 ,
       75.476137  , 82.27616535, 82.27616535, 81.2896791 , 81.2896791 ,
       85.58565401, 82.08947368, 82.08947368, 75.86330179, 75.59371728,
       75.38987474, 76.5528974 , 74.54479918, 75.33472367, 75.33472367,
       75.8837454 , 75.56096285, 75.42506527, 74.35061602, 77.03272532,
       77.03272532, 77.40474906, 77.40474906, 81.14755043])

# 자동차 분포


In [ ]:
자동차 = gpd.read_file("/gdrive/My Drive/광양시 데이터/03.광양시_자동차등록현황_격자(100X100).geojson")
자동차

,id,totale,geometry
0,146,0,"POLYGON ((127.52550 35.07616, 127.52640 35.076..."
1,147,0,"POLYGON ((127.52550 35.07543, 127.52640 35.075..."
2,148,0,"POLYGON ((127.52550 35.07469, 127.52640 35.074..."
3,149,0,"POLYGON ((127.52550 35.07396, 127.52640 35.073..."
4,150,0,"POLYGON ((127.52550 35.07322, 127.52640 35.073..."
...,...,...,...
69954,117103,0,"POLYGON ((127.78602 35.01218, 127.78691 35.012..."
69955,117104,0,"POLYGON ((127.78602 35.01144, 127.78691 35.011..."
69956,117105,0,"POLYGON ((127.78602 35.01071, 127.78691 35.010..."
69957,117106,0,"POLYGON ((127.78602 35.00997, 127.78691 35.009..."


In [ ]:
자동차.describe()

,id,totale
count,69959.000000,69959.000000
mean,58268.185537,0.226733
std,27381.578314,3.459083
min,146.000000,0.000000
25%,37285.500000,0.000000
50%,56760.000000,0.000000
75%,78881.500000,0.000000
max,117107.000000,246.000000


In [ ]:
자동차_경위도 = np.stack([자동차['geometry'].centroid.x, 자동차['geometry'].centroid.y], axis=1).reshape([1, -1, 2])

In [ ]:
for i in range(len(주차장)):
    주차장_경위도 = np.stack([주차장.lon,주차장.lat], axis=1).reshape([-1,1,2])
    조건 = np.sum((주차장_경위도 - 자동차_경위도)**2, axis=2)**0.5 < 거리*축척
    자동차분포 = np.array([자동차.loc[x,'totale'].mean() for x in 조건])

In [ ]:
자동차분포

array([0.28992033, 0.27675985, 0.28747802, 0.29653692, 0.29620188,
       0.30014928, 0.29064588, 0.29744694, 0.29372985, 0.28650998,
       0.29729566, 0.26003369, 0.28746597, 0.29592145, 0.30507583,
       0.29986022, 0.29430258, 0.28249719, 0.28886924, 0.30488137,
       0.29832257, 0.28050211, 0.28050211, 0.2750267 , 0.2750267 ,
       0.30410352, 0.2890571 , 0.2890571 , 0.31666125, 0.31456326,
       0.31070065, 0.32440321, 0.29118838, 0.3116407 , 0.3116407 ,
       0.31824859, 0.31658526, 0.31437306, 0.28654352, 0.26521956,
       0.26521956, 0.26971289, 0.26971289, 0.2757973 ])